In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)


/Users/conda/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/conda/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/conda/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/conda/opt/anaconda3/envs/te

1.13.1
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/10
60000/60000 [==============================] - 12s 206us/sample - loss: 0.1573 - acc: 0.9532
Epoch 2/10
60000/60000 [==============================] - 14s 231us/sample - loss: 0.0538 - acc: 0.9834
Epoch 3/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.0342 - acc: 0.9896
Epoch 4/10
60000/60000 [==============================] - 14s 230us/sample - loss: 0.0230 - acc: 0.9929
Epoch 5/10
60000/60000 [==============================] - 14s 235us/sample - loss: 0.0165 - acc: 0.9948
Epoch 6/10
60000/60000 [==============================] - 15s 247us/sample - loss: 0.0139 - acc: 0.9956
Epoch 7/10
60000/60000 [==============================] - 17s 283us/sample - loss: 0.0090 - acc: 0.9971
Epoch 8/10
53824/60000 [=========================>....] - ETA: 1s - loss: 0.0065 - acc: 0.9979